## Beyond `__syncthreads()`: The Cooperative Groups API

Cooperative Groups provides a flexible, composable API for thread synchronization and collective operations.

In [ ]:
%%writefile cg_basics_advanced.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

__global__ void cgDemo() {
    // Get thread block group
    cg::thread_block block = cg::this_thread_block();
    
    // Partition into tiles of 32 threads (warp-aligned)
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    // Partition into tiles of 16 threads
    cg::thread_block_tile<16> half_warp = cg::tiled_partition<16>(block);
    
    // Get coalesced threads (active threads in warp)
    cg::coalesced_group active = cg::coalesced_threads();
    
    if (block.thread_rank() == 0) {
        printf("Block size: %d\n", block.size());
        printf("Warp tile size: %d\n", warp.size());
        printf("Half-warp tile size: %d\n", half_warp.size());
    }
}

int main() {
    cgDemo<<<1, 128>>>();
    cudaDeviceSynchronize();
    return 0;
}

In [ ]:
!nvcc cg_basics_advanced.cu -o cg_basics_advanced && ./cg_basics_advanced

## CG Collective Operations: `reduce()`

Hardware-accelerated reductions with `cg::reduce()` - much faster than manual shuffle reductions!

In [ ]:
%%writefile cg_reduce.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

__global__ void cgReduceKernel(int* input, int* output, int n) {
    cg::thread_block block = cg::this_thread_block();
    cg::thread_block_tile<32> warp = cg::tiled_partition<32>(block);
    
    int tid = block.thread_rank();
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Load value
    int val = (idx < n) ? input[idx] : 0;
    
    // Warp-level reduction using CG (hardware-accelerated on CC 8.0+)
    int warp_sum = cg::reduce(warp, val, cg::plus<int>());
    
    // First thread in each warp has the sum
    __shared__ int warp_sums[32];  // Max 32 warps per block
    if (warp.thread_rank() == 0) {
        warp_sums[tid / 32] = warp_sum;
    }
    block.sync();
    
    // Final reduction of warp sums (first warp only)
    if (tid < 32) {
        int num_warps = (blockDim.x + 31) / 32;
        val = (tid < num_warps) ? warp_sums[tid] : 0;
        int block_sum = cg::reduce(warp, val, cg::plus<int>());
        
        if (tid == 0) {
            output[blockIdx.x] = block_sum;
        }
    }
}

int main() {
    const int N = 1024;
    int h_input[N], h_output[8];
    
    for (int i = 0; i < N; i++) h_input[i] = 1;  // Sum should be N
    
    int *d_input, *d_output;
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_output, 8 * sizeof(int));
    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);
    
    cgReduceKernel<<<8, 128>>>(d_input, d_output, N);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_output, d_output, 8 * sizeof(int), cudaMemcpyDeviceToHost);
    
    int total = 0;
    for (int i = 0; i < 8; i++) total += h_output[i];
    printf("Sum of %d ones = %d (expected %d)\n", N, total, N);
    
    cudaFree(d_input);
    cudaFree(d_output);
    return 0;
}

In [ ]:
!nvcc cg_reduce.cu -o cg_reduce && ./cg_reduce

## CG Scan Operations: `inclusive_scan()` and `exclusive_scan()`

In [ ]:
%%writefile cg_scan.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/scan.h>

namespace cg = cooperative_groups;

__global__ void cgScanKernel(int* input, int* incl_output, int* excl_output, int n) {
    cg::thread_block block = cg::this_thread_block();
    
    int tid = block.thread_rank();
    int val = (tid < n) ? input[tid] : 0;
    
    // Inclusive scan: each element includes itself
    // Input:  [1, 2, 3, 4]
    // Output: [1, 3, 6, 10]
    int incl = cg::inclusive_scan(block, val, cg::plus<int>());
    
    // Exclusive scan: each element excludes itself
    // Input:  [1, 2, 3, 4]
    // Output: [0, 1, 3, 6]
    int excl = cg::exclusive_scan(block, val, cg::plus<int>());
    
    if (tid < n) {
        incl_output[tid] = incl;
        excl_output[tid] = excl;
    }
}

int main() {
    const int N = 8;
    int h_input[N] = {1, 2, 3, 4, 5, 6, 7, 8};
    int h_incl[N], h_excl[N];
    
    int *d_input, *d_incl, *d_excl;
    cudaMalloc(&d_input, N * sizeof(int));
    cudaMalloc(&d_incl, N * sizeof(int));
    cudaMalloc(&d_excl, N * sizeof(int));
    
    cudaMemcpy(d_input, h_input, N * sizeof(int), cudaMemcpyHostToDevice);
    
    cgScanKernel<<<1, N>>>(d_input, d_incl, d_excl, N);
    cudaDeviceSynchronize();
    
    cudaMemcpy(h_incl, d_incl, N * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_excl, d_excl, N * sizeof(int), cudaMemcpyDeviceToHost);
    
    printf("Input:          ");
    for (int i = 0; i < N; i++) printf("%3d ", h_input[i]);
    printf("\nInclusive scan: ");
    for (int i = 0; i < N; i++) printf("%3d ", h_incl[i]);
    printf("\nExclusive scan: ");
    for (int i = 0; i < N; i++) printf("%3d ", h_excl[i]);
    printf("\n");
    
    cudaFree(d_input);
    cudaFree(d_incl);
    cudaFree(d_excl);
    return 0;
}

In [ ]:
!nvcc cg_scan.cu -o cg_scan && ./cg_scan

## Warp-Aggregated Atomics Pattern

Reduce atomic contention by having one thread per warp perform the atomic operation.

In [ ]:
%%writefile cg_warp_atomics.cu
#include <stdio.h>
#include <cooperative_groups.h>
#include <cooperative_groups/reduce.h>

namespace cg = cooperative_groups;

// Naive: every thread does atomic - HIGH CONTENTION
__device__ int naiveAtomicInc(int* counter) {
    return atomicAdd(counter, 1);
}

// Optimized: warp-aggregated atomic - MUCH FASTER
__device__ int warpAggregatedAtomicInc(int* counter) {
    cg::coalesced_group active = cg::coalesced_threads();
    
    int warp_res;
    if (active.thread_rank() == 0) {
        // Only leader does the atomic
        warp_res = atomicAdd(counter, active.size());
    }
    
    // Broadcast result and compute individual values
    warp_res = active.shfl(warp_res, 0);
    return warp_res + active.thread_rank();
}

__global__ void testNaive(int* counter, int* results, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        results[idx] = naiveAtomicInc(counter);
    }
}

__global__ void testWarpAgg(int* counter, int* results, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        results[idx] = warpAggregatedAtomicInc(counter);
    }
}

int main() {
    const int N = 1000000;
    int *d_counter, *d_results;
    
    cudaMalloc(&d_counter, sizeof(int));
    cudaMalloc(&d_results, N * sizeof(int));
    
    // Benchmark naive
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaMemset(d_counter, 0, sizeof(int));
    cudaEventRecord(start);
    testNaive<<<(N+255)/256, 256>>>(d_counter, d_results, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float naive_ms;
    cudaEventElapsedTime(&naive_ms, start, stop);
    
    // Benchmark warp-aggregated
    cudaMemset(d_counter, 0, sizeof(int));
    cudaEventRecord(start);
    testWarpAgg<<<(N+255)/256, 256>>>(d_counter, d_results, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float warp_ms;
    cudaEventElapsedTime(&warp_ms, start, stop);
    
    printf("Naive atomics:          %.3f ms\n", naive_ms);
    printf("Warp-aggregated atomics: %.3f ms\n", warp_ms);
    printf("Speedup: %.1fx\n", naive_ms / warp_ms);
    
    cudaFree(d_counter);
    cudaFree(d_results);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    
    return 0;
}

In [ ]:
!nvcc cg_warp_atomics.cu -o cg_warp_atomics && ./cg_warp_atomics

## Grid-Level Synchronization

Cooperative kernel launch allows synchronizing ALL threads across the entire grid.

In [ ]:
%%writefile cg_grid_sync.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include <cooperative_groups.h>

namespace cg = cooperative_groups;

__global__ void gridSyncKernel(int* data, int n) {
    cg::grid_group grid = cg::this_grid();
    
    int idx = grid.thread_rank();
    
    // Phase 1: Each thread writes its index
    if (idx < n) {
        data[idx] = idx;
    }
    
    // Grid-wide synchronization - ALL threads wait here
    grid.sync();
    
    // Phase 2: Read neighbor's value (guaranteed to be written)
    if (idx < n && idx > 0) {
        data[idx] += data[idx - 1];
    }
    
    if (idx == 0) {
        printf("Grid sync completed across %lu threads\n", grid.size());
    }
}

int main() {
    // Check for cooperative launch support
    int dev = 0;
    int supportsCoopLaunch = 0;
    cudaDeviceGetAttribute(&supportsCoopLaunch, cudaDevAttrCooperativeLaunch, dev);
    
    if (!supportsCoopLaunch) {
        printf("Device does not support cooperative launch\n");
        return 0;
    }
    
    const int N = 1024;
    int* d_data;
    cudaMalloc(&d_data, N * sizeof(int));
    
    // Use cooperative launch
    void* args[] = {&d_data, (void*)&N};
    
    int numBlocks;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(&numBlocks, gridSyncKernel, 128, 0);
    
    int numSMs;
    cudaDeviceGetAttribute(&numSMs, cudaDevAttrMultiProcessorCount, dev);
    
    dim3 grid(numSMs * numBlocks);
    dim3 block(128);
    
    cudaLaunchCooperativeKernel((void*)gridSyncKernel, grid, block, args);
    cudaDeviceSynchronize();
    
    int h_data[N];
    cudaMemcpy(h_data, d_data, N * sizeof(int), cudaMemcpyDeviceToHost);
    printf("Results: data[0]=%d, data[1]=%d, data[10]=%d\n",
           h_data[0], h_data[1], h_data[10]);
    
    cudaFree(d_data);
    return 0;
}

In [ ]:
!nvcc cg_grid_sync.cu -o cg_grid_sync && ./cg_grid_sync

## CG Reduction Operators

| Operator | Returns |
|----------|--------|
| `cg::plus<T>()` | Sum |
| `cg::less<T>()` | Minimum |
| `cg::greater<T>()` | Maximum |
| `cg::bit_and<T>()` | Bitwise AND |
| `cg::bit_or<T>()` | Bitwise OR |
| `cg::bit_xor<T>()` | Bitwise XOR |

## Key Takeaways

1. **`cg::reduce()`** - Hardware-accelerated on CC 8.0+
2. **`cg::inclusive_scan()` / `cg::exclusive_scan()`** - Parallel prefix operations
3. **Warp-aggregated atomics** - Reduce contention dramatically
4. **Grid sync** - Requires `cudaLaunchCooperativeKernel`
5. **Flexible partitioning** - `tiled_partition<N>()` for any power-of-2